# 1. Dựng UI bằng tool
# 2. Dựng Fullstack UI trên server
# 3. Test API Backend riêng trước + đồng thời test riêng UI (giả input, output). 
# 4. Test tích hợp





# Kiểm tra API Backend với Bot ID 115

Để kiểm tra xem API Backend có hoạt động đúng không với Bot ID 115, tôi sẽ cung cấp một script Python đơn giản để gửi yêu cầu WebSocket với các thông số bạn đã cung cấp.

## Script kiểm tra WebSocket

```python
import asyncio
import websockets
import json
import uuid
import time

async def test_websocket():
    client_id = f"test-client-{uuid.uuid4()}"
    uri = f"ws://localhost:25050/ws/{client_id}"
    
    print(f"Connecting to {uri}...")
    
    try:
        async with websockets.connect(uri) as websocket:
            print(f"Connected successfully!")
            
            # Gửi yêu cầu bắt đầu cuộc hội thoại với Bot ID 115
            request = {
                "type": "start_conversation",
                "agent_mode": "bot_id",
                "bot_id": 115,  # Bot ID 115 theo yêu cầu
                "user_prompts": [
                    {
                        "id": 1,
                        "content": "I am Cuong"  # Prompt User role A
                    }
                ],
                "initial_conversation_history": [
                    {"role": "roleA", "content": "sẵn sàng"}
                ],
                "max_turns": 3
            }
            
            print(f"Sending request with Bot ID 115...")
            await websocket.send(json.dumps(request))
            print(f"Request sent!")
            
            # Nhận và hiển thị phản hồi
            message_count = 0
            start_time = time.time()
            
            while True:
                try:
                    response = await websocket.recv()
                    data = json.loads(response)
                    message_count += 1
                    
                    message_type = data.get("type", "unknown")
                    print(f"\n[{message_count}] Received message type: {message_type}")
                    
                    if message_type == "conversations_created":
                        print(f"Conversation IDs: {[c['id'] for c in data['conversations']]}")
                    
                    elif message_type == "status":
                        print(f"Status: {data.get('message', '')}")
                    
                    elif message_type == "message":
                        msg = data.get("message", {})
                        print(f"Message from {msg.get('role')}: {msg.get('content')[:100]}...")
                        if len(msg.get('content', '')) > 100:
                            print("...")
                        print(f"Response time: {msg.get('response_time')} seconds")
                    
                    elif message_type == "error":
                        print(f"ERROR: {data.get('message', '')}")
                    
                    elif message_type == "completion":
                        print(f"Conversation completed: {data.get('message', '')}")
                        print(f"Total messages received: {message_count}")
                        print(f"Total time: {time.time() - start_time:.2f} seconds")
                        break
                    
                except websockets.exceptions.ConnectionClosed:
                    print("Connection closed")
                    break
                except Exception as e:
                    print(f"Error: {str(e)}")
                    break
            
            print("\nTest completed!")
    
    except Exception as e:
        print(f"Failed to connect: {str(e)}")

if __name__ == "__main__":
    print("Starting WebSocket test with Bot ID 115...")
    asyncio.run(test_websocket())
```

## Hướng dẫn sử dụng

1. Lưu script trên vào một file, ví dụ: `test_bot_115.py`
2. Đảm bảo Backend đang chạy trên cổng 25050
3. Cài đặt thư viện cần thiết:
   ```bash
   pip install websockets
   ```
4. Chạy script:
   ```bash
   python test_bot_115.py
   ```

## Kết quả mong đợi

Nếu Backend hoạt động đúng, bạn sẽ thấy các thông báo sau:

1. Kết nối thành công
2. Nhận thông báo khởi tạo cuộc hội thoại
3. Nhận thông báo trạng thái "Đang khởi tạo cuộc hội thoại với Bot ID: 115..."
4. Nhận tin nhắn ban đầu "sẵn sàng" từ User
5. Nhận thông báo trạng thái "Đang chờ phản hồi từ bot..."
6. Nhận tin nhắn phản hồi từ Bot
7. Tiếp tục các lượt hội thoại
8. Nhận thông báo hoàn thành cuộc hội thoại

## Kiểm tra logs

Trong khi chạy script, bạn nên mở một terminal khác để theo dõi logs của Backend:

```bash
tail -f WebSimulationConversation/backend/logs/api.log
```

Điều này sẽ giúp bạn xác định bất kỳ vấn đề nào xảy ra trong quá trình xử lý.

## Nếu gặp lỗi

Nếu bạn gặp lỗi khi khởi tạo cuộc hội thoại với Bot ID 115, hãy thử:

1. Kiểm tra xem Bot ID 115 có tồn tại và hoạt động không
2. Kiểm tra logs để xem lỗi cụ thể
3. Thử lại với Bot ID 16 (đã được xác nhận hoạt động) để so sánh

Nếu Bot ID 115 không hoạt động, bạn sẽ nhận được thông báo lỗi từ Backend, và bạn có thể cần liên hệ với người quản lý API bên ngoài để kiểm tra.








# Quy trình xử lý cuộc hội thoại trong Backend

Tôi hiểu rõ hơn về quy trình xử lý cuộc hội thoại trong Backend. Dưới đây là phân tích chi tiết:

## Quy trình xử lý đầy đủ

1. **Khởi tạo cuộc hội thoại**:
   - Frontend gửi yêu cầu khởi tạo cuộc hội thoại qua WebSocket
   - Backend khởi tạo cuộc hội thoại với API bên ngoài (Bot ID)
   - Backend gửi thông báo khởi tạo thành công đến Frontend

2. **Tin nhắn ban đầu**:
   - Backend gửi tin nhắn ban đầu (thường là "sẵn sàng") đến API bên ngoài
   - API bên ngoài trả về phản hồi đầu tiên từ Role B

3. **Xử lý phản hồi từ Role B**:
   - Backend nhận phản hồi từ API bên ngoài
   - Backend gửi phản hồi này đến Frontend để hiển thị
   - **Đồng thời**, Backend sử dụng phản hồi này làm đầu vào cho OpenAI để tạo phản hồi tiếp theo của Role A

4. **Tạo phản hồi cho Role A**:
   - Backend sử dụng OpenAI để tạo phản hồi cho Role A dựa trên lịch sử cuộc hội thoại
   - Backend gửi phản hồi này đến Frontend để hiển thị
   - Backend gửi phản hồi này đến API bên ngoài để lấy phản hồi tiếp theo từ Role B

5. **Lặp lại bước 3-4** cho đến khi đạt số lượt tối đa

## Luồng dữ liệu chi tiết

```
Frontend                   Backend                         API bên ngoài            OpenAI
   |                          |                                |                       |
   | Khởi tạo cuộc hội thoại  |                                |                       |
   |------------------------->|                                |                       |
   |                          | Khởi tạo cuộc hội thoại        |                       |
   |                          |------------------------------->|                       |
   |                          |<-------------------------------|                       |
   |<-------------------------|                                |                       |
   |                          |                                |                       |
   |                          | Gửi tin nhắn ban đầu           |                       |
   |                          |------------------------------->|                       |
   |                          |                                |                       |
   |                          | Nhận phản hồi từ Role B        |                       |
   |                          |<-------------------------------|                       |
   |                          |                                |                       |
   | Hiển thị phản hồi Role B |                                |                       |
   |<-------------------------|                                |                       |
   |                          |                                |                       |
   |                          | Gửi lịch sử hội thoại          |                       |
   |                          |------------------------------------------------------>|
   |                          |                                |                       |
   |                          | Nhận phản hồi cho Role A       |                       |
   |                          |<------------------------------------------------------|
   |                          |                                |                       |
   | Hiển thị phản hồi Role A |                                |                       |
   |<-------------------------|                                |                       |
   |                          |                                |                       |
   |                          | Gửi phản hồi Role A            |                       |
   |                          |------------------------------->|                       |
   |                          |                                |                       |
   |                          | Nhận phản hồi từ Role B        |                       |
   |                          |<-------------------------------|                       |
   |                          |                                |                       |
   | Hiển thị phản hồi Role B |                                |                       |
   |<-------------------------|                                |                       |
   |                          |                                |                       |
   |        ... Lặp lại cho đến khi đạt số lượt tối đa ...    |                       |
```

## Điểm quan trọng

1. **Xử lý đồng thời**:
   - Backend xử lý phản hồi từ Role B và gửi đến Frontend
   - **Đồng thời**, Backend sử dụng phản hồi này làm đầu vào cho OpenAI

2. **Luồng dữ liệu liên tục**:
   - Mỗi phản hồi từ Role B được gửi ngay đến Frontend
   - Mỗi phản hồi từ Role A được gửi ngay đến Frontend
   - Không có sự chậm trễ giữa việc nhận phản hồi và hiển thị

3. **Xử lý bất đồng bộ**:
   - Backend sử dụng asyncio để xử lý các yêu cầu bất đồng bộ
   - Điều này cho phép Backend gửi thông báo trạng thái trong khi đang chờ phản hồi

## Cải thiện hiệu suất

Để cải thiện hiệu suất và trải nghiệm người dùng:

1. **Gửi thông báo trạng thái**:
   ```python
   status_message = {
       "type": "status",
       "conversation_id": conversation_id,
       "message": "Đang xử lý phản hồi từ bot..."
   }
   await manager.send_message(status_message, client_id)
   ```

2. **Xử lý phản hồi từng phần**:
   ```python
   # Nếu API hỗ trợ streaming
   async for chunk in api_client.send_message_streaming(message):
       chunk_message = {
           "type": "message_chunk",
           "conversation_id": conversation_id,
           "message": {
               "role": "Agent",
               "content": chunk,
               "response_time": time.time() - start_time
           }
       }
       await manager.send_message(chunk_message, client_id)
   ```

3. **Hiển thị trạng thái "đang nhập"**:
   ```python
   typing_message = {
       "type": "typing",
       "conversation_id": conversation_id,
       "role": "Agent",
       "is_typing": True
   }
   await manager.send_message(typing_message, client_id)
   ```

## Kết luận

Backend của bạn đã được thiết kế để xử lý cuộc hội thoại một cách hiệu quả, với phản hồi từ Role B được gửi đến Frontend và đồng thời được sử dụng làm đầu vào cho OpenAI để tạo phản hồi tiếp theo của Role A.

Việc sử dụng WebSocket cho phép Backend gửi dữ liệu theo thời gian thực đến Frontend, tạo ra trải nghiệm người dùng mượt mà và tự nhiên. Mỗi tin nhắn được hiển thị ngay khi có sẵn, không có sự chậm trễ giữa việc nhận phản hồi và hiển thị.


```mermaid
sequenceDiagram
    participant User as Người dùng
    participant Frontend
    participant Backend
    participant ExternalAPI as API Bên ngoài
    participant OpenAI

    User->>Frontend: Nhập prompt và chọn Bot ID
    User->>Frontend: Nhấn "Start Simulation"
    Frontend->>Backend: Gửi yêu cầu qua WebSocket
    Backend->>Frontend: Trả về conversation_id
    
    Note over Backend, Frontend: Kết nối WebSocket vẫn mở

    Backend->>Frontend: Gửi thông báo "Đang khởi tạo cuộc hội thoại..."
    Backend->>ExternalAPI: Gửi yêu cầu khởi tạo (bất đồng bộ)
    
    alt Khởi tạo thành công
        ExternalAPI-->>Backend: Trả về kết quả khởi tạo
        Backend->>Frontend: Gửi thông báo "Đã khởi tạo thành công"
        
        Backend->>Frontend: Gửi tin nhắn người dùng đầu tiên
        Backend->>Frontend: Gửi thông báo "Đang chờ phản hồi từ bot..."
        Backend->>ExternalAPI: Gửi tin nhắn người dùng (bất đồng bộ)
        ExternalAPI-->>Backend: Trả về phản hồi của Role B
        Backend->>Frontend: Gửi phản hồi của Role B
        
        loop Cho mỗi lượt hội thoại
            Backend->>Frontend: Gửi tin nhắn người dùng tiếp theo
            Backend->>Frontend: Gửi thông báo "Đang chờ phản hồi từ bot..."
            Backend->>ExternalAPI: Gửi tin nhắn người dùng (bất đồng bộ)
            ExternalAPI-->>Backend: Trả về phản hồi của Role B
            Backend->>Frontend: Gửi phản hồi của Role B
            
            Backend->>OpenAI: Gửi lịch sử hội thoại đến OpenAI
            OpenAI-->>Backend: Trả về phản hồi cho Role A
            Backend->>Frontend: Gửi phản hồi của Role A
        end
        
        Backend->>Frontend: Gửi thông báo hoàn thành cuộc hội thoại
    else Khởi tạo thất bại
        ExternalAPI-->>Backend: Trả về lỗi hoặc timeout
        Backend->>Frontend: Gửi thông báo lỗi với gợi ý sử dụng Bot ID 16
    end
    
    Frontend->>User: Hiển thị toàn bộ cuộc hội thoại

```